In [2]:
api_key = 'a5d04411cecb64b2e73e2bc896da553e'

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pprint import pprint
import csv, sqlite3

# Import API key
#from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [6]:
capitals = pd.read_csv('db/capitalscsv.csv')
capitals.head()

,Capital,State
0,Montgomery,Alabama
1,Juneau,Alaska
2,Phoenix,Arizona
3,Little Rock,Arkansas
4,Sacramento,California


In [7]:
capitals['Capital']

0         Montgomery
1             Juneau
2            Phoenix
3        Little Rock
4         Sacramento
5             Denver
6           Hartford
7              Dover
8        Tallahassee
9            Atlanta
10          Honolulu
11             Boise
12       Springfield
13      Indianapolis
14        Des Moines
15            Topeka
16         Frankfort
17       Baton Rouge
18           Augusta
19         Annapolis
20            Boston
21           Lansing
22        Saint Paul
23           Jackson
24    Jefferson City
25            Helena
26           Lincoln
27       Carson City
28           Concord
29           Trenton
30          Santa Fe
31            Albany
32           Raleigh
33          Bismarck
34          Columbus
35     Oklahoma City
36             Salem
37        Harrisburg
38        Providence
39          Columbia
40            Pierre
41         Nashville
42            Austin
43    Salt Lake City
44        Montpelier
45          Richmond
46           Olympia
47        Cha

In [8]:
#url
url = "http://api.openweathermap.org/data/2.5/weather?"

query_url = f"{url}appid={api_key}&units=imperial&q="

In [9]:
#run a test to see data and [u]
test3 = requests.get(query_url + 'New York').json()
test3
#gotta dig into dic
#response['name']
#response['clouds']['all']
#response['sys']['country']
#response['dt']
#response['main']['temp']
#response['coord']['lon']
#response['coord']['lat']
#response['main']['temp_max']
#response['wind']['speed']

{'coord': {'lon': -73.99, 'lat': 40.73},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 75.38,
  'pressure': 1013,
  'humidity': 41,
  'temp_min': 68,
  'temp_max': 82.4},
 'visibility': 16093,
 'wind': {'speed': 6.93, 'deg': 280},
 'clouds': {'all': 1},
 'dt': 1562034964,
 'sys': {'type': 1,
  'id': 4686,
  'message': 0.0174,
  'country': 'US',
  'sunrise': 1561973294,
  'sunset': 1562027461},
 'timezone': -14400,
 'id': 5128581,
 'name': 'New York',
 'cod': 200}

In [10]:
capital_df = pd.DataFrame({'City' : capitals['Capital']})
capital_df['Cloudiness'] = ''
capital_df['Country'] = ''
capital_df['Date'] = ''
capital_df['Humidity'] = ''
capital_df['Lat'] = ''
capital_df['Lng'] = ''
capital_df['Max Speed'] = ''
capital_df['Wind Speed'] = ''
capital_df['Temp (F)'] = ''
row_count = 1
capital_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Speed,Wind Speed,Temp (F)
0,Montgomery,,,,,,,,,
1,Juneau,,,,,,,,,
2,Phoenix,,,,,,,,,
3,Little Rock,,,,,,,,,
4,Sacramento,,,,,,,,,


In [15]:
print('Executing Data Retrieval')
for index, row in capital_df.iterrows():
    city = row['City']
    response = requests.get(query_url + city).json()
    
    if response["cod"] == "404":
        print('City is no bueno... skipping city...')
        capital_df.loc[index, 'Country'] = 'Drop'
    else: 

        print(f' Processing Record {row_count} for {city}')

        capital_df.loc[index, 'Cloudiness'] = float(response['clouds']['all'])
        capital_df.loc[index, 'Country'] = (response['sys']['country'])
        capital_df.loc[index, 'Date'] = response['dt']
        capital_df.loc[index, 'Humidity'] = float(response['main']['humidity'])
        capital_df.loc[index, 'Lat'] = float(response['coord']['lat'])
        capital_df.loc[index, 'Lng'] = float(response['coord']['lon'])
        capital_df.loc[index, 'Max Speed'] = float(response['main']['temp_max'])
        capital_df.loc[index, 'Wind Speed'] = float(response['wind']['speed'])
        capital_df.loc[index, 'Temp (F)'] = float(response["main"]["temp"])
        row_count = row_count + 1
#        weather_df[weather_df.name != city]
#        row_count = row_count + 1

Executing Data Retrieval
 Processing Record 51 for Montgomery
 Processing Record 52 for Juneau
 Processing Record 53 for Phoenix
 Processing Record 54 for Little Rock
 Processing Record 55 for Sacramento
 Processing Record 56 for Denver
 Processing Record 57 for Hartford
 Processing Record 58 for Dover
 Processing Record 59 for Tallahassee
 Processing Record 60 for Atlanta
 Processing Record 61 for Honolulu
 Processing Record 62 for Boise
 Processing Record 63 for Springfield
 Processing Record 64 for Indianapolis
 Processing Record 65 for Des Moines
 Processing Record 66 for Topeka
 Processing Record 67 for Frankfort
 Processing Record 68 for Baton Rouge
 Processing Record 69 for Augusta
 Processing Record 70 for Annapolis
 Processing Record 71 for Boston
 Processing Record 72 for Lansing
 Processing Record 73 for Saint Paul
 Processing Record 74 for Jackson
 Processing Record 75 for Jefferson City
 Processing Record 76 for Helena
 Processing Record 77 for Lincoln
 Processing Record 7

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [16]:
#capital_df =  capital_df[capital_df.Country != 'Drop']

In [17]:
capital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
City          50 non-null object
Cloudiness    50 non-null object
Country       50 non-null object
Date          50 non-null object
Humidity      50 non-null object
Lat           50 non-null object
Lng           50 non-null object
Max Speed     50 non-null object
Wind Speed    50 non-null object
Temp (F)      50 non-null object
dtypes: object(10)
memory usage: 4.0+ KB


In [18]:
capital_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Speed,Wind Speed,Temp (F)
0,Montgomery,1,US,1562035945,94,32.37,-86.3,82.4,4.29,79.56
1,Juneau,20,US,1562035891,60,58.3,-134.42,71.6,10.29,70.57
2,Phoenix,1,US,1562035649,9,33.45,-112.08,105.8,14.99,102.45
3,Little Rock,1,US,1562035945,74,34.75,-92.29,82.4,3.36,78.49
4,Sacramento,1,US,1562035746,34,38.58,-121.49,87.8,6.93,78.93
5,Denver,75,US,1562035724,72,39.74,-104.98,71.01,16.11,67.44
6,Hartford,1,US,1562035657,56,41.76,-72.69,73.4,5.5,67.33
7,Dover,1,US,1562035943,88,39.16,-75.52,73.4,3.27,69.03
8,Tallahassee,20,US,1562035861,74,30.44,-84.28,86,8.05,82.9
9,Atlanta,1,US,1562035810,74,33.75,-84.39,82.4,8.7,78.44


In [19]:
capital_df.to_csv("db/Clean_Capital_Weather.csv")